In [ ]:
# -*- coding: utf-8 -*-

#import talib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn import datasets
from pylab import mpl
import pandas

In [ ]:
company_name = 'GOOG' # stock code
path = './data/' + company_name # file path
duration = 365 # duration for total training sets
future_duration = 1 # future duration to determine labels


In [ ]:
# data processing
# get historical price
all_price = pandas.read_csv(path + '_history.csv', header = 0, sep = ',', names = ['date', 'volume', 'close', 'high', 'open', 'low'])
price = all_price.tail(duration).reset_index(drop = True)
#print price

# get ema
all_ema = pandas.read_csv(path + '_ema.csv', header = 0, sep = ',')
ema = all_ema.tail(duration).head(duration - 1).reset_index(drop = True)
#print ema

#get macd
all_macd = pandas.read_csv(path + '_macd.csv', header = 0, sep = ',')
macd = all_macd.tail(duration).head(duration - 1).reset_index(drop = True)
#print macd

#get obv
all_obv = pandas.read_csv(path + '_obv.csv', header = 0, sep = ',')
obv = all_obv.tail(duration).head(duration - 1).reset_index(drop = True)
#print obv

#get rsi
all_rsi = pandas.read_csv(path + '_rsi.csv', header = 0, sep = ',')
rsi = all_rsi.tail(duration).head(duration - 1).reset_index(drop = True)
#print rsi

#get stoch
all_stoch = pandas.read_csv(path + '_stoch.csv', header = 0, sep = ',')
stoch = all_stoch.tail(duration).head(duration - 1).reset_index(drop = True)
#print stoch



In [ ]:
# ema, macd, obv, rsi, #stoch
features = np.stack((ema.EMA.values, macd.MACD.values, obv.OBV.values, rsi.CCI.values), axis = 1)
print len(features)
print features

In [ ]:
labels = []
for i in range(duration -future_duration):
    labels.append(1 if price.close[i + future_duration] >= price.close[i] else 0)
print len(labels)
print labels
#print price [['date','close']]

In [ ]:
# SVM in TensorFlow

np.random.seed(1)
tf.set_random_seed(1)

sess=tf.Session()

x_vals=features
y_vals=labels

train_test_split = np.random.rand(len(features)) < 0.70 # randomly divide training sets and test sets
x_vals_train = features[train_test_split]
y_vals_train = np.array(labels)[train_test_split]
x_vals_test = features[~train_test_split]
y_vals_test = np.array(labels)[~train_test_split]

# batch
batch_size = 1
x_data = tf.placeholder(shape=[None, 4], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)
W = tf.Variable(tf.random_normal(shape=[4,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

model_output=tf.matmul(x_data,W)+b
l2_norm = tf.reduce_sum(tf.square(W))
# regularization
alpha = tf.constant([0.1])
# loss function
classification_term = tf.reduce_mean(tf.maximum(0.,1.-model_output*y_target))
loss = classification_term+alpha*l2_norm
# output
prediction = tf.sign(model_output)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_target),tf.float32))
train_step=tf.train.GradientDescentOptimizer(0.001).minimize(loss)
# train
sess.run(tf.global_variables_initializer())
loss_vec = []
train_accuracy = []
test_accuracy = []
for i in range(duration):
    rand_index = np.random.choice(len(x_vals_train), size=batch_size)
    rand_x = x_vals_train[rand_index]
    rand_y = np.transpose([y_vals_train[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target:rand_y})
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss)
    train_acc_temp = sess.run(accuracy, feed_dict={x_data: x_vals_train, y_target: np.transpose([y_vals_train])})
    train_accuracy.append(train_acc_temp)
    test_acc_temp = sess.run(accuracy, feed_dict={x_data: x_vals_test, y_target: np.transpose([y_vals_test])})
    test_accuracy.append(test_acc_temp)
    if (i+1)%100==0:
        print('Step #' + str(i+1) + ' W = ' + str(sess.run(W)) + 'b = ' + str(sess.run(b)))
        print('Loss = ' + str(test_acc_temp))
plt.plot(loss_vec)
plt.plot(train_accuracy)
plt.plot(test_accuracy)
plt.legend(['Loss','Train Accuracy','Test Accuracy'])
plt.ylim(0.,1.)
